In [1]:
import pandas as pd
import config
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from binance.client import Client

# Initialize the Binance client with your API key and secret
client = Client(api_key= config.BINANCEAPI, api_secret=config.BINANACESEC)

# Retrieve the historical klines data for BTC/USDT
klines = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, "1 week ago UTC+2")
data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

# Convert timestamp to datetime and set it as index
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
data.set_index('timestamp', inplace=True)

# Convert close column to numeric data type
data['close'] = pd.to_numeric(data['close'])

# Drop rows containing missing values
data.dropna(inplace=True)

# Calculate momentum
data['momentum'] = data['close'].pct_change()

# Define profit-taking and stop-loss percentages
take_profit = 0.02
stop_loss = -0.01

# Loop through buy and sell signals to calculate trades and outcomes
trades = []
for i in range(1, len(data)):
    if data['momentum'][i-1] < 0 and data['momentum'][i] > 0:
        # Buy signal
        entry_price = data['close'][i]
    elif data['momentum'][i-1] > 0 and data['momentum'][i] < 0:
        # Sell signal
        exit_price = data['close'][i]
        pnl = (exit_price - entry_price) / entry_price
        if pnl >= take_profit:
            outcome = 'Win'
        elif pnl <= stop_loss:
            outcome = 'Loss'
        else:
            outcome = 'Neutral'
        trades.append({'entry_time': data.index[i-1], 'exit_time': data.index[i], 'entry_price': entry_price, 'exit_price': exit_price, 'pnl': pnl, 'outcome': outcome})

# Calculate win rate and PNL
total_trades = len(trades)
winning_trades = len([t for t in trades if t['outcome'] == 'Win'])
losing_trades = len([t for t in trades if t['outcome'] == 'Loss'])
neutral_trades = len([t for t in trades if t['outcome'] == 'Neutral'])
win_rate = winning_trades / total_trades
total_pnl = sum([t['pnl'] for t in trades])
average_pnl = total_pnl / total_trades

# Create subplots to show momentum and buying/selling markers
figure = make_subplots(rows=2, cols=1)
figure.add_trace(go.Scatter(x=data.index, y=data['close'], name='Close Price'))
figure.add_trace(go.Scatter(x=data.index, y=data['momentum'], name='Momentum', yaxis='y2'))

# Add buy and sell signals
figure.add_trace(go.Scatter(x=data.loc[data['momentum'] > 0].index, y=data.loc[data['momentum'] > 0]['close'], mode='markers', name='Buy', marker=dict(color='green', symbol='triangle-up')))
figure.add_trace(go.Scatter(x=data.loc[data['momentum'] < 0].index, y=data.loc[data['momentum'] < 0]['close'], mode='markers', name='Sell', marker=dict(color='red', symbol='triangle-down')))

# Add trades
if trades:
    for i, trade in enumerate(trades):
        figure.add_annotation(x=trade['entry_time'], y=trade['entry_price'], text=f'Trade {i+1}', showarrow=True, arrowhead=1, arrowcolor='blue')
        figure.add_annotation(x=trade['exit_time'], y=trade['exit_price'], text=f'{trade["outcome"]}: {trade["pnl"]:.2%}', showarrow=True, arrowhead=1, arrowcolor='blue')

# Set layout
figure.update_layout(title='Algorithmic Trading using Momentum Strategy', xaxis_title='Date', yaxis_title='Price')
figure.update_yaxes(title="Momentum", secondary_y=True)

# Add win rate and PNL to layout
figure.update_layout(
    annotations=[
        dict(
            text=f'Total PNL: {total_pnl:.2%}',
            xref='paper', yref='paper',
            x=0, y=1, showarrow=False,
            align='right',
            bgcolor='white',
            font=dict(size=16)
        )
    ]
)

figure.show()

In [26]:
total_trades, winning_trades, losing_trades

(323, 0, 0)